In [1]:
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

In [18]:
continuous_after = pd.read_csv("..\dataset\demographics_to_placement_simulation_WR_no_cap_continuous_2.csv")

unknown_employment_index = continuous_after[ (continuous_after['employmentStatus_Full_Time'] == 0) & (continuous_after['employmentStatus_Part_Time'] == 0) & (continuous_after['employmentStatus_Unemployed'] == 0)].index
unknown_employment_age = continuous_after[ (continuous_after['employmentStatus_Part_Time'] == 0) & (continuous_after['employmentStatus_Unemployed'] == 0) & (continuous_after['employmentStatus_Full_Time'] == 0) ]['age']
unemployment_index = continuous_after[ continuous_after['employmentStatus_Unemployed'] == 1].index
unknown_employment_index

type1_index = list(unemployment_index) + list(unknown_employment_index)
type2_index = list(continuous_after[ continuous_after['employmentStatus_Full_Time'] == 1].index) + list(continuous_after[ continuous_after['employmentStatus_Part_Time'] == 1].index)
new_row = [0] * continuous_after.shape[0]
for index in type1_index:
    new_row[index] = 1

continuous_after['Employed'] = new_row
continuous_after.drop(['employmentStatus_Part_Time', 'employmentStatus_Full_Time'], axis =1, inplace = True )

new_row = [0] * continuous_after.shape[0]
for index in type2_index:
    new_row[index] = 1

continuous_after['Unemployed'] = new_row
continuous_after.drop(['employmentStatus_Unemployed'], axis =1, inplace = True )
continuous_after.drop(['Unemployed','ocp_wr_low_los', 'ocp_jl_low_los', 'ocp_hd_low_los','licenseStatus_Not_Suspended','licenseStatus_Suspended'], axis=1, inplace=True) 

#Cost multiplication
continuous_after['weighted_jl_total'] = continuous_after['c_occu_jail']*continuous_after['ocp_jl_total']
continuous_after['weighted_wr_total'] = continuous_after['c_occu_wr']*continuous_after['ocp_wr_total']
continuous_after['weighted_hd_total'] = continuous_after['c_occu_hd']*continuous_after['ocp_hd_total']
continuous_after.drop(['c_occu_jail', 'c_occu_wr', 'c_occu_hd', 'ocp_jl_total', 'ocp_wr_total', 'ocp_hd_total'], axis = 1, inplace=True)

# one hot encoding  -> the target is "placement", so we don't need to encode this column, also bcz RandomForest can handle categorical target data
# work release = 0 | home detention = 1 | jail = 2
for ind, item in continuous_after.iterrows():
    if item['placement'] == 'work release':
        continuous_after['placement'].iat[ind] = '0'
    elif item['placement'] == 'home detention':
        continuous_after['placement'].iat[ind] = '1'
    elif item['placement'] == 'jail':
        continuous_after['placement'].iat[ind] = '2'

#object to int
continuous_after['placement'] = pd.to_numeric(continuous_after['placement'])

# drop imbalanced data
continuous_after.drop(['race_Other', 'registeredSexOffender_TRUE', 'violentOffender_TRUE', 'gangMember_x_TRUE', 'homeless_TRUE'], axis = 1, inplace = True)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
y3 = continuous_after["placement"]
X3 = continuous_after.drop(["placement"], axis = 1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split( X3, y3, train_size = 0.7, random_state = 40) 

In [21]:
rf_model3 = RandomForestClassifier(max_depth= 7, max_features = 'sqrt', min_samples_leaf = 5, 
 min_samples_split = 5, n_estimators = 100, random_state= 50) 
rf_model3.fit(X_train_3, y_train_3) 

RandomForestClassifier(max_depth=7, min_samples_leaf=5, min_samples_split=5,
                       random_state=50)

In [237]:
wr_sorted = sorted(continuous_after['weighted_wr_total'], reverse= True)
top_5_percent_index = int(len(wr_sorted) * 0.05)
top_5_percent = wr_sorted[:top_5_percent_index]
# 0.430848 -> 0.40550400000000003 | min 0.0057, mid: 0.1028

In [239]:
hd_sorted = sorted(continuous_after['weighted_hd_total'], reverse= True)
top_5_percent_index_hd = int(len(hd_sorted) * 0.05)
top_5_percent_hd = hd_sorted[:top_5_percent_index_hd]
# 0.137808 -> 0.133056 | min: 0.0005, mid: 0.0148

In [241]:
jl_sorted = sorted(continuous_after['weighted_jl_total'], reverse= True)
top_5_percent_index_jl = int(len(jl_sorted) * 0.05)
top_5_percent_jl = jl_sorted[:top_5_percent_index_jl]
# 0.39888 -> 0.36432000000000003 | min: 0.12, mid: 0.1195

In [245]:
# these are the indexes for row with congested occupancy | c.o are the top 0.1% value for each category
index_congested_wr = []
index_congested_hd = []
index_congested_jl = []
for i, row in continuous_after.iterrows():
    if (row['weighted_hd_total'] >= 0.40550400000000003):
        index_congested_hd.append(i)
    if (row['weighted_wr_total'] >= 0.133056):
        index_congested_wr.append(i)
    if (row['weighted_jl_total'] >= 0.36432000000000003):
        index_congested_jl.append(i)

In [66]:
import lime
import lime.lime_tabular
import numpy as np

In [269]:
explainer2 = lime.lime_tabular.LimeTabularExplainer(X_train_3, feature_names=list(continuous_after.columns.drop(['placement'])), class_names=np.array([0,1,2]), discretize_continuous=False)

In [291]:
## wr
count_wr = 0
count_hd = 0
count_jl = 0
available_index = []
satisfied_index = []
for i in index_congested_wr:
    if i in list(X_test_3.index):
        available_index.append(i)
        # take explanation object
        exp2 = explainer2.explain_instance(np.array(X_test_3[X_test_3.index == i])[0], rf_model3.predict_proba, num_features=14, top_labels=3)
        # extract predicted class label
        pred_class = list(exp2.predict_proba).index(max(exp2.predict_proba))
        # extract the feature contribution according to the class label
        feature_contribution = exp2.as_list(label = pred_class)
        # sorted the feature contribution in a decreasing order
        sorted_contribution = sorted(feature_contribution, key=lambda x: abs(x[1]), reverse= True)
        # check if among the top 5 features, there are any occupancy features.
        checked_list = [x[0] for x in sorted_contribution[:5] if ('weighted_jl_total' in x[0]) or ('weighted_hd_total' in x[0]) or ('weighted_wr_total' in x[0])]
        for occupancy_feature in checked_list:
            satisfied_index.append(i)
            if 'weighted_jl_total' in occupancy_feature:
                count_jl = count_jl + 1
            if 'weighted_hd_total' in occupancy_feature:
                count_hd = count_hd + 1
            if 'weighted_wr_total' in occupancy_feature:
                count_wr = count_wr + 1

c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:4

In [306]:
print(f"Number of rows that have occupancy features in their top 5 importance: {len(satisfied_index)}")
print(f"Number of rows that have weighted_wr_total in their top 5: {count_wr}")
print(f"Number of rows that have weighted_hd_total in their top 5: {count_hd}")
print(f"Number of rows that have weighted_jl_total in their top 5: {count_jl}")

Number of rows that have occupancy features in their top 5 importance: 314
Number of rows that have weighted_wr_total in their top 5: 215
Number of rows that have weighted_hd_total in their top 5: 35
Number of rows that have weighted_jl_total in their top 5: 64


In [289]:
## hd
count2_wr = 0
count2_hd = 0
count2_jl = 0
available_index2 = []
satisfied_index2 = []
for i in index_congested_hd:
    if i in list(X_test_3.index):
        available_index2.append(i)
        # take explanation object
        exp2 = explainer2.explain_instance(np.array(X_test_3[X_test_3.index == i])[0], rf_model3.predict_proba, num_features=14, top_labels=3)
        # extract predicted class label
        pred_class = list(exp2.predict_proba).index(max(exp2.predict_proba))
        # extract the feature contribution according to the class label
        feature_contribution = exp2.as_list(label = pred_class)
        # sorted the feature contribution in a decreasing order
        sorted_contribution = sorted(feature_contribution, key=lambda x: abs(x[1]), reverse= True)
        # check if among the top 5 features, there are any occupancy features.
        checked_list = [x[0] for x in sorted_contribution[:5] if ('weighted_jl_total' in x[0]) or ('weighted_hd_total' in x[0]) or ('weighted_wr_total' in x[0])]
        for occupancy_feature in checked_list:
            satisfied_index2.append(i)
            if 'weighted_jl_total' in occupancy_feature:
                count2_jl = count2_jl + 1
            if 'weighted_hd_total' in occupancy_feature:
                count2_hd = count2_hd + 1
            if 'weighted_wr_total' in occupancy_feature:
                count2_wr = count2_wr + 1

In [305]:
print(f"Number of rows that have occupancy features in their top 5 importance: {len(satisfied_index2)}")
print(f"Number of rows that have weighted_wr_total in their top 5: {count2_wr}")
print(f"Number of rows that have weighted_hd_total in their top 5: {count2_hd}")
print(f"Number of rows that have weighted_jl_total in their top 5: {count2_jl}")

Number of rows that have occupancy features in their top 5 importance: 0
Number of rows that have weighted_wr_total in their top 5: 0
Number of rows that have weighted_hd_total in their top 5: 0
Number of rows that have weighted_jl_total in their top 5: 0


In [292]:
## jl
count3_wr = 0
count3_hd = 0
count3_jl = 0
available_index3 = [] 
satisfied_index3 = []
for i in index_congested_jl:
    if i in list(X_test_3.index):
        available_index3.append(i)
        # take explanation object
        exp2 = explainer2.explain_instance(np.array(X_test_3[X_test_3.index == i])[0], rf_model3.predict_proba, num_features=14, top_labels=3)
        # extract predicted class label
        pred_class = list(exp2.predict_proba).index(max(exp2.predict_proba))
        # extract the feature contribution according to the class label
        feature_contribution = exp2.as_list(label = pred_class)
        # sorted the feature contribution in a decreasing order
        sorted_contribution = sorted(feature_contribution, key=lambda x: abs(x[1]), reverse= True)
        # check if among the top 5 features, there are any occupancy features.
        checked_list = [x[0] for x in sorted_contribution[:5] if ('weighted_jl_total' in x[0]) or ('weighted_hd_total' in x[0]) or ('weighted_wr_total' in x[0])]
        for occupancy_feature in checked_list:
            satisfied_index3.append(i)
            if 'weighted_jl_total' in occupancy_feature:
                count3_jl = count3_jl + 1
            if 'weighted_hd_total' in occupancy_feature:
                count3_hd = count3_hd + 1
            if 'weighted_wr_total' in occupancy_feature:
                count3_wr = count3_wr + 1

c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\voqua\anaconda3\Lib\site-packages\sklearn\base.py:4

In [304]:
print(f"Number of rows that have occupancy features in their top 5 importance: {len(satisfied_index3)}")
print(f"Number of rows that have weighted_wr_total in their top 5: {count3_wr}")
print(f"Number of rows that have weighted_hd_total in their top 5: {count3_hd}")
print(f"Number of rows that have weighted_jl_total in their top 5: {count3_jl}")

Number of rows that have occupancy features in their top 5 importance: 24
Number of rows that have weighted_wr_total in their top 5: 13
Number of rows that have weighted_hd_total in their top 5: 2
Number of rows that have weighted_jl_total in their top 5: 9
